# PySpark & PyDeequ Shakeout


In [1]:
%%bash
pip install pydeequ

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession, Row, DataFrame

import sagemaker_pyspark
import pydeequ

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

Please set env variable SPARK_VERSION


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-72f00b16-ce0a-40d6-86ca-2f00b504d91d;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in local-m2-cache
	found org.scalanlp#breeze_2.12;0.13.2 in local-m2-cache
	found org.scalanlp#breeze-macros_2.12;0.13.2 in local-m2-cache
	found org.scala-lang#scala-reflect;2.12.1 in local-m2-cache
	found com.github.fommil.netlib#core;1.1.2 in spark-list
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.

22/11/23 19:44:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/23 19:44:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/23 19:44:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/23 19:44:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/23 19:44:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
%%bash

echo $JAVA_HOME
$JAVA_HOME/bin/java -version

/home/ec2-user/anaconda3/envs/JupyterSystemEnv


openjdk version "11.0.1" 2018-10-16 LTS
OpenJDK Runtime Environment Zulu11.2+3 (build 11.0.1+13-LTS)
OpenJDK 64-Bit Server VM Zulu11.2+3 (build 11.0.1+13-LTS, mixed mode)


In [4]:
# Read a CSV file from local instance directory into a Spark dataframe
file_input = 'data/TOP_FedClct_20221001_20221001.csv'

df = spark.read \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load(file_input) 

df.printSchema()
df.show(5)

# Write a copy of the Spark dataframe to a Parquet output file
file_output = file_input.replace('.csv', '.parquet')
df.write.mode('overwrite').parquet(file_output)
print(f'File output: {file_output}')


root
 |-- Record Date: timestamp (nullable = true)
 |-- Creditor Agency Name: string (nullable = true)
 |-- Agency ID: integer (nullable = true)
 |-- Agency Site Type Code: string (nullable = true)
 |-- Agency Site ID: string (nullable = true)
 |-- Agency Site Name: string (nullable = true)
 |-- Source Agency Alpha: string (nullable = true)
 |-- Source Agency Numeric: string (nullable = true)
 |-- Source Agency Site ID: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Payment Agency ID: string (nullable = true)
 |-- Agency Type Code: string (nullable = true)
 |-- Payment Source Description: string (nullable = true)
 |-- Payment Source Code: string (nullable = true)
 |-- Payment Category Description: string (nullable = true)
 |-- Payment Type Description: string (nullable = true)
 |-- Payment Type Code: string (nullable = true)
 |-- Payment Type ID: integer (nullable = true)
 |-- Debt Type Code: string (nullable = true)
 |-- Debt Type Description: string (nullabl

File output: data/TOP_FedClct_20221001_20221001.parquet


In [5]:
# Read a Parquet file from local instance directory
file_input = 'data/TOP_FedClct_20221001_20221001.parquet'

df = spark.read.parquet(file_input) 
df.printSchema()
df.show(5)


root
 |-- Record Date: timestamp (nullable = true)
 |-- Creditor Agency Name: string (nullable = true)
 |-- Agency ID: integer (nullable = true)
 |-- Agency Site Type Code: string (nullable = true)
 |-- Agency Site ID: string (nullable = true)
 |-- Agency Site Name: string (nullable = true)
 |-- Source Agency Alpha: string (nullable = true)
 |-- Source Agency Numeric: string (nullable = true)
 |-- Source Agency Site ID: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Payment Agency ID: string (nullable = true)
 |-- Agency Type Code: string (nullable = true)
 |-- Payment Source Description: string (nullable = true)
 |-- Payment Source Code: string (nullable = true)
 |-- Payment Category Description: string (nullable = true)
 |-- Payment Type Description: string (nullable = true)
 |-- Payment Type Code: string (nullable = true)
 |-- Payment Type ID: integer (nullable = true)
 |-- Debt Type Code: string (nullable = true)
 |-- Debt Type Description: string (nullabl

In [6]:
%%bash
# copy local CSV file to S3 using AWS CLI
aws s3 cp data/TOP_FedClct_20221001_20221001.csv s3://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv
    
aws s3 ls s3://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv    

upload: data/TOP_FedClct_20221001_20221001.csv to s3://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv
2022-11-23 19:44:22     920049 TOP_FedClct_20221001_20221001.csv


In [7]:
# Read the CSV file from S3 using Python AWS SDK
import boto3
s3 = boto3.client('s3')

s3_bucket = 'wc2h-dtl-dev-datawork'
s3_key = 'rmyers07/data/TOP_FedClct_20221001_20221001.csv'

s3_obj = s3.get_object(Bucket=s3_bucket, Key=s3_key )
data = s3_obj['Body'].read().decode('utf-8')

for n,line in enumerate(data.split('\n')):
    print(line)
    if n > 5:
        break


Record Date,Creditor Agency Name,Agency ID,Agency Site Type Code,Agency Site ID,Agency Site Name,Source Agency Alpha,Source Agency Numeric,Source Agency Site ID,Agency Name,Payment Agency ID,Agency Type Code,Payment Source Description,Payment Source Code,Payment Category Description,Payment Type Description,Payment Type Code,Payment Type ID,Debt Type Code,Debt Type Description,Taxable Indicator,Total Gross Amount,Total Net Amount,Total Gross Count,Total Net Count,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
2022-10-01,Bureau of the Fiscal Service,83,Federal,33MK,33-MK-BUREAU OF FISCAL SERVICE,33,51,MK,Internal Revenue Service,132,Federal,Tax Refund,TAX,TDO,Tax Refund EFT,IE,12,FN,FEDERAL NON-TAX,N,986.00,986.00,1,1,2023,1,2022,4,10,01
2022-10-01,Bureau of the Fiscal Service,83,Federal,33M7,33-M7-FISCAL SERVICE - DMSC,33,1C,M7,Department of Homeland Security,21,State,Vendor payment,VEN,TDO,Vendor EFT,VE,13,FN,FEDERAL N

In [8]:
# Read the CSV file from S3 into a Spark data frame
s3_url_input = 's3a://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv'

df = spark.read \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load(s3_url_input) 

df.show(5)


22/11/23 19:44:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/11/23 19:44:24 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv.
org.apache.hadoop.fs.s3a.AWSBadRequestException: getFileStatus on s3a://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv: com.amazonaws.services.s3.model.AmazonS3Exception: The provided token is malformed or otherwise invalid. (Service: Amazon S3; Status Code: 400; Error Code: InvalidToken; Request ID: AAJW6XTJ98PRMM1F; S3 Extended Request ID: CnTZRfDUIxCOv7EVP3/PdxqTA0Bu6yMu3Y+1DltUYRhySDTc+pwfvGS7PLOFcxplXzjtBydZB+E=; Proxy: null), S3 Extended Request ID: CnTZRfDUIxCOv7EVP3/PdxqTA0Bu6yMu3Y+1DltUYRhySDTc+pwfvGS7PLOFcxplXzjtBydZB+E=:InvalidToken: The provided token is malformed or otherwise invalid. (Service: Amazon S3; Status

Py4JJavaError: An error occurred while calling o51.load.
: org.apache.hadoop.fs.s3a.AWSBadRequestException: getFileStatus on s3a://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: FD3S30JKVRYGYVEY; S3 Extended Request ID: adfdllci1MHP4I80x4FTJT4C2GrxkrMZbKeGRURZk/XahCsJWF/yiZNJ5B34gUjqUOQR7NLbzzU=; Proxy: null), S3 Extended Request ID: adfdllci1MHP4I80x4FTJT4C2GrxkrMZbKeGRURZk/XahCsJWF/yiZNJ5B34gUjqUOQR7NLbzzU=:400 Bad Request: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: FD3S30JKVRYGYVEY; S3 Extended Request ID: adfdllci1MHP4I80x4FTJT4C2GrxkrMZbKeGRURZk/XahCsJWF/yiZNJ5B34gUjqUOQR7NLbzzU=; Proxy: null)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:243)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.s

In [9]:
%%bash
# Use AWS CLI to list objects in publicly-accessible S3 parquet data set (AWS US East Region)
aws s3 ls s3://amazon-reviews-pds/parquet/product_category=Electronics/

# WC2H fails , DAAB LAB works 


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


CalledProcessError: Command 'b'# Use AWS CLI to list objects in publicly-accessible S3 parquet data set (AWS US East Region)\naws s3 ls s3://amazon-reviews-pds/parquet/product_category=Electronics/\n\n# WC2H fails , DAAB LAB works \n'' returned non-zero exit status 255.

In [10]:
# Read a publicly-accessible S3 parquet data set into a Spark dataframe (AWS US East Region)
df = spark.read.parquet("s3a://amazon-reviews-pds/parquet/product_category=Electronics/")
df.printSchema()
df.show(5)

# WC2H fails , DAAB LAB works 

22/11/23 19:45:41 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://amazon-reviews-pds/parquet/product_category=Electronics/.
org.apache.hadoop.fs.s3a.AWSBadRequestException: getFileStatus on s3a://amazon-reviews-pds/parquet/product_category=Electronics: com.amazonaws.services.s3.model.AmazonS3Exception: The provided token is malformed or otherwise invalid. (Service: Amazon S3; Status Code: 400; Error Code: InvalidToken; Request ID: XCK6EDDR4PK13KMT; S3 Extended Request ID: IAoh8StamstsSJFIXMpuT+fkAW2/4fkyfl1fnHSUCDw2fOXEegRZCbbT2Xez/hJfToYqTlYR3cClg9fajHY+CQ==; Proxy: null), S3 Extended Request ID: IAoh8StamstsSJFIXMpuT+fkAW2/4fkyfl1fnHSUCDw2fOXEegRZCbbT2Xez/hJfToYqTlYR3cClg9fajHY+CQ==:InvalidToken: The provided token is malformed or otherwise invalid. (Service: Amazon S3; Status Code: 400; Error Code: InvalidToken; Request ID: XCK6EDDR4PK13KMT; S3 Extended Request ID: IAoh8StamstsSJFIXMpuT+fkAW2/4fkyfl1fnHSUCDw2fOXEegRZCb

Py4JJavaError: An error occurred while calling o54.parquet.
: org.apache.hadoop.fs.s3a.AWSBadRequestException: getFileStatus on s3a://amazon-reviews-pds/parquet/product_category=Electronics: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: 0PZZKCKGXGBFWJ40; S3 Extended Request ID: 4zZBiQ5h8tYDEVBgfAk/8WiJqbh6ZT/lrgAFd1jAJGlnhrztRgOLh4N3lb1i0wp/x49zxxFpQEctikVMAg8VAw==; Proxy: null), S3 Extended Request ID: 4zZBiQ5h8tYDEVBgfAk/8WiJqbh6ZT/lrgAFd1jAJGlnhrztRgOLh4N3lb1i0wp/x49zxxFpQEctikVMAg8VAw==:400 Bad Request: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: 0PZZKCKGXGBFWJ40; S3 Extended Request ID: 4zZBiQ5h8tYDEVBgfAk/8WiJqbh6ZT/lrgAFd1jAJGlnhrztRgOLh4N3lb1i0wp/x49zxxFpQEctikVMAg8VAw==; Proxy: null)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:243)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	